In [ ]:
from pathlib import Path

TEST_ROOT = Path().cwd().parent.parent / "tests"
TEST_DIR = TEST_ROOT / "test_data"

In [ ]:
from pymatgen.core.structure import Structure

si_structure = Structure.from_file(TEST_DIR / "structures" / "Si.cif")

In [ ]:
from jobflow import run_locally

from atomate2.forcefields.flows.phonons import PhononMaker

flow = PhononMaker(
    min_length=3.0,
    born_maker=None,
    use_symmetrized_structure="conventional",
    create_thermal_displacements=False,
    store_force_constants=False,
    prefer_90_degrees=False,
    generate_frequencies_eigenvectors_kwargs={"tstep": 100},
).make(si_structure)
run_locally(flow, create_folders=True, raise_immediately=True)

One can switch to a different force field as well!

In [ ]:
from atomate2.forcefields.jobs import ForceFieldRelaxMaker, ForceFieldStaticMaker

flow = PhononMaker(
    min_length=3.0,
    use_symmetrized_structure="conventional",
    create_thermal_displacements=False,
    store_force_constants=False,
    prefer_90_degrees=False,
    generate_frequencies_eigenvectors_kwargs={"tstep": 100},
    static_energy_maker=ForceFieldStaticMaker(force_field_name="MACE"),
    bulk_relax_maker=ForceFieldRelaxMaker(force_field_name="MACE"),
    phonon_displacement_maker=ForceFieldStaticMaker(force_field_name="MACE"),
).make(si_structure)

run_locally(flow, create_folders=True, raise_immediately=True)